In [102]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')


RendererRegistry.enable('default')

Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [103]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [104]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [105]:
refract = 'c5ac5e60-7da0-429f-98e8-19ccdd77d835'
refract_cluna = '5b0257c3-e93b-49d4-93f6-e370bc3b3f50'
lping = '879d0260-93a9-4576-a1bd-2cc3dc26bf13'
ystaking = '3ff0fc49-5a0d-4cdf-a8ab-33f8ea7e755f'
swaps = 'a3b3ad75-15b2-4a40-9850-00e8c1c33ef0'
router = '69b149fb-81ba-4860-aac1-b17f0d6d7688'

lping_df = claim(lping)
refract_df = claim(refract)
refract_cluna_df = claim(refract_cluna)
ystaking_df = claim(ystaking)
swaps_df = claim(swaps)
router_df = claim(router)

lping_df.to_csv('../data/lping.csv')
refract_df.to_csv('../data/refract.csv')
refract_cluna_df.to_csv('../data/refract_cluna.csv')
ystaking_df.to_csv('../data/ystaking.csv')
swaps_df.to_csv('../data/swaps.csv')
router_df.to_csv('../data/router.csv')

In [106]:
lping_df = pd.read_csv('../data/lping.csv', index_col=0)
refract_df = pd.read_csv('../data/refract.csv', index_col=0)
refract_cluna_df = pd.read_csv('../data/refract_cluna.csv', index_col=0)
ystaking_df = pd.read_csv('../data/ystaking.csv', index_col=0)
swaps_df = pd.read_csv('../data/swaps.csv', index_col=0)
router_df = pd.read_csv('../data/router.csv', index_col=0)

### LPing

In [107]:
lping_df['action'] = lping_df.apply(lambda row: row['0_action'] if row['0_action'] == 'provide_liquidity' else row['1_action'],axis=1)

In [108]:
provide_ = lping_df[lping_df.action=='provide_liquidity']
withdraw_ = lping_df[lping_df.action=='withdraw_liquidity']

In [109]:
provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['3_amount'],axis=1)
provide_['yluna_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['2_amount'],axis=1)
provide_['sender'] = provide_['from_']
provide_['hr'] = provide_.block_timestamp.str[:-9] + '00:00.000'
provide_['day'] = provide_.block_timestamp.str[:-13]
provide_ = provide_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
provide_['amount_signed'] = provide_.yluna_amount
provide_

,block_timestamp,sender,tx_id,action,prism_amount,yluna_amount,hr,day,amount_signed
0,2022-02-15 08:37:52.327,terra1jnl320gtknwhy8h8hur867vyx4rdrlafn0g9xr,B4E4FBCEE5F1E470FF3EBCA6D8542B3451665F62CC29AFCBBFF10C89248288AC,provide_liquidity,1822.258838,178.380000,2022-02-15 08:00:00.000,2022-02-15,178.380000
1,2022-02-15 03:00:27.844,terra1yu076nvjakk759r9tlsnhhtvnyn3addq60ush0,384FE38C67923B4BB26C0E41B39F6374E73599FDA310D35A13D8AC9C4F6B1F29,provide_liquidity,6069.849705,745.500000,2022-02-15 03:00:00.000,2022-02-15,745.500000
2,2022-02-15 05:01:12.665,terra1k0ymmku570mmjzuqqzu54m56v0cry2srgq86an,4C76FDF33EA0D93A77A4D4C86C514F4889786333E65C1FD09B3B0C8DB920B00B,provide_liquidity,161.685638,18.552790,2022-02-15 05:00:00.000,2022-02-15,18.552790
4,2022-02-15 14:50:23.547,terra1evehddfs29vaey6zspamavl3xuafsate6g6kr9,B80FD56F02A42CFDDF5D9490B007DC23CBDB9246004973C2BF790895BB9F1C45,provide_liquidity,4.190515,0.460000,2022-02-15 14:00:00.000,2022-02-15,0.460000
5,2022-02-15 08:14:06.743,terra1smgnuccp3987efl75dmlgndwq58kplmvj0zvgn,88C8FEB836543DD9CD42427838E85C68B27345B17FA9537826FB92F75C7D88DD,provide_liquidity,155.097614,15.470000,2022-02-15 08:00:00.000,2022-02-15,15.470000
...,...,...,...,...,...,...,...,...,...
1363,2022-02-23 02:14:27.778,terra1y945wth8wmycp0ewl95cwdppcg648xgc5ls680,90476EB84853022831993A1BE02ADCD6DD853B12CE6BFC2968CE8FC6AC61740D,provide_liquidity,10.876907,0.999999,2022-02-23 02:00:00.000,2022-02-23,0.999999
1365,2022-02-23 05:46:49.640,terra1jtmpyx3wdpswham92nzatwpwc6h4ulkv2xrdaa,FBE8F931CE74184412ACB559B6974F36D9ABAEACB265476525F4A9DBA0467880,provide_liquidity,63692.231467,5755.595788,2022-02-23 05:00:00.000,2022-02-23,5755.595788
1367,2022-02-17 11:20:14.966,terra1sg0f2wv908allekgp4fxf7xstufndq3atwkpwn,F2F10F5698685D188A2200AD506FEDE265F1EC00CC7D34BD93C7926B16B8FDB2,provide_liquidity,50.671553,5.249999,2022-02-17 11:00:00.000,2022-02-17,5.249999
1371,2022-02-25 20:38:34.689,terra1gwm4s5460gxx0qptl7tf9u00uxr5r0zw07sj54,88DB7D65CED5B9C17A5891B5F37192C95F911208894903F9324D7BACA0CFE552,provide_liquidity,10.699599,0.848801,2022-02-25 20:00:00.000,2022-02-25,0.848801


In [110]:
pd.set_option("display.max_colwidth", 400)
withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['2_amount'],axis=1)
withdraw_['yluna_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['1_amount'],axis=1)
withdraw_['sender'] = withdraw_['from_']
withdraw_['hr'] = withdraw_.block_timestamp.str[:-9] + '00:00.000'
withdraw_['day'] = withdraw_.block_timestamp.str[:-13]
withdraw_ = withdraw_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
withdraw_['amount_signed'] = -withdraw_.yluna_amount
withdraw_

,block_timestamp,sender,tx_id,action,prism_amount,yluna_amount,hr,day,amount_signed
3,2022-02-15 14:47:48.048,terra18xe77cuksxc4u9e43kmp87nvfnr92qlsj6u3kp,1B88C29683899F3F892C564E7926042219414714A0EE82D5476919AF500E123D,withdraw_liquidity,237.052216,2.726577,2022-02-15 14:00:00.000,2022-02-15,-2.726577
7,2022-02-15 06:36:54.229,terra1n6qjrvw27su7xewe6e0vstk4tvayk3shzp7r00,0FC4C750B4FCCA6E2559BAD38B8A8B9CDC238057A9D8B6158586C1DB034AD006,withdraw_liquidity,1477.735890,17.238019,2022-02-15 06:00:00.000,2022-02-15,-17.238019
10,2022-02-15 15:53:05.800,terra1hhchj94dzn3yaxmvk4atd3rnfycly3cswwa4nu,C7F5508305DB499E441BE83D25D9FA7B92ABC189827C2C6CEE33C7010A6BBC46,withdraw_liquidity,124.382697,1.535129,2022-02-15 15:00:00.000,2022-02-15,-1.535129
19,2022-02-15 22:12:17.334,terra1tnhw5lhqe2qum0j2kzza2nplrm43x0ardhhgnw,29CCBE55AF923D5C56BD93F586F31A115FB755886755F2BF57E8C437449ECB87,withdraw_liquidity,1093.818702,14.469449,2022-02-15 22:00:00.000,2022-02-15,-14.469449
22,2022-02-20 11:36:44.279,terra1efq832jsdclrxfw2hh2dqlxgk5u0nx76s8uwd3,3A779705BCC51DC4A08D0409ECD51DDA8D49FBEEA38BB2E41B9C0189872A6A5E,withdraw_liquidity,633.970664,5.875611,2022-02-20 11:00:00.000,2022-02-20,-5.875611
...,...,...,...,...,...,...,...,...,...
1369,2022-02-17 16:17:08.352,terra1csxqptsve542l3d37gftnud40xyflwqg7lrvy6,DEA08437B0EDFA6B6E3637AB84F9BE89F0CAD797F6007955409C27E43EE647E5,withdraw_liquidity,20534.981349,202.086201,2022-02-17 16:00:00.000,2022-02-17,-202.086201
1370,2022-02-25 11:31:01.955,terra14xkdtqapr6q03scq2ulnra5dwpd5jazctkyn2v,BC1C2EA63FD504833602F007F22CD87181D423DF838DEDF225551BEB7FACD535,withdraw_liquidity,2513.330271,15.688001,2022-02-25 11:00:00.000,2022-02-25,-15.688001
1373,2022-03-05 11:10:50.140,terra1lrpcl54ypnd2aqu0cg0wd2r9j9kylm5hf9ehg0,0F77EB82BA7DEA6C20DD207CBBC03EC16F45032A51B9A8315E3F80AEEEEA309F,withdraw_liquidity,826.560908,5.190306,2022-03-05 11:00:00.000,2022-03-05,-5.190306
1374,2022-03-01 00:51:28.996,terra1mn9cj7w6r8numdg84zl2v3jr0794f5tyu0hxu8,63CFDAB1CDE844996F5C33CAFAFB3489727791FE04684429E02285B6C0CF62A6,withdraw_liquidity,460.889259,2.225184,2022-03-01 00:00:00.000,2022-03-01,-2.225184


In [111]:
all_lps = withdraw_.append(provide_)
daily_delta_lp = all_lps.groupby('day').amount_signed.sum().reset_index()
daily_delta_lp = daily_delta_lp.sort_values(by='day')
daily_delta_lp['cumsum'] = daily_delta_lp.amount_signed.cumsum().apply(lambda x: round(x,2))
daily_delta_lp.columns = ['Time', 'Amount signed', 'Amount']
daily_delta_lp['Type'] = 'yLuna LP'
daily_delta_lp.head()

,Time,Amount signed,Amount,Type
0,2022-02-14,30295.717889,30295.72,yLuna LP
1,2022-02-15,19098.750501,49394.47,yLuna LP
2,2022-02-16,-243.645779,49150.82,yLuna LP
3,2022-02-17,-5670.718499,43480.10,yLuna LP
4,2022-02-18,-5750.253495,37729.85,yLuna LP


### Refract

In [112]:
refract_cluna_df.head()

,0_action,0_amount,1_amount,1_contract_address,2_amount,2_contract_address,block_timestamp,event_index,from_,msg_index,to_,tx_id
0,transfer_from,180.000000,180.000000,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,180.000000,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 13:24:17.166,1,terra1xc6kttvedkf96f9rz0klc3d8ff7sfy9hjrxg6v,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,349C9F2F540E2C0B39034F5B8D5BBC114E468C71150A6E19FE3FE09551D3461E
1,transfer_from,0.018917,0.018917,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,0.018917,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 16:29:56.862,1,terra1jxaq9xnympm5l4yvccdayyara65uvphu6l8tva,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,D6644B621DD9A31DBD0800BC6D9D5889ED5EEB861DF7023519DA3E0B1AF41C7C
2,transfer_from,265.542139,265.542139,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,265.542139,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 15:21:18.039,1,terra1dyu2rvykn5wkhp2jwa90a9fkqwk88z62mk47nf,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,D5C7595087DBFEDFE0F7A31E424FDAEE004B63228617395A31EB755628AA8B6F
3,transfer_from,1.841609,1.841609,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,1.841609,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 16:59:19.788,1,terra10xt0yhdullk2n8st46dja4tazkrckkyvlsjfac,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,ED9B70CD5DC09AA1DA555929ADCC8D173C1E6C01CFECB62776B5DE639EC8B21D
4,transfer_from,1.000000,1.000000,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,1.000000,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 15:33:56.849,1,terra1mkhgc2tq33n90uaa5nc4yrhgp86xy79e8mtxz5,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,6B6BE138D1F9C2079687DD316C5F1C7E9928E1C93314121CA68C771F674670F9


In [113]:
refract_cluna_df['user'] = refract_cluna_df['from_']

In [114]:
refract_cluna_df.shape

(1767, 13)

In [115]:
refract_cluna_df[refract_cluna_df.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']

,0_action,0_amount,1_amount,1_contract_address,2_amount,2_contract_address,block_timestamp,event_index,from_,msg_index,to_,tx_id,user
881,transfer_from,6.028406,6.028406,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,6.028406,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-28 18:25:39.031,1,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,A28152A997953C3F845E5A1A7DAF86312FAA301392DE1778B63DE9128CFE246E,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q
1000,transfer_from,5.172802,5.172802,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,5.172802,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-28 19:47:31.424,1,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,E5024C01400407E1C538E1969FDE71C6CA43F49A71D1B649BC672D13BB9DDC29,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q
1163,transfer_from,10.233401,10.233401,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,10.233401,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-24 19:41:51.724,1,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,A05AC252DA3FB4090AB88BFCC08232591674C54480AFC62A0A6695589F8B1D6C,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q
1571,transfer_from,5.114376,5.114376,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,5.114376,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-23 07:48:24.069,1,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,1,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,5CDF87523A2D6EB5EDFE1FB549232B5F4EC2CA979EE719D92CAF0DA64A394071,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q


In [116]:
refract_cluna_df_pol = refract_cluna_df[['block_timestamp','tx_id','user','0_action','0_amount']]
refract_cluna_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']

In [117]:
refract_cluna_df_pol['asset_given'] = 'cLUNA'
refract_cluna_df_pol['asset_received'] = 'yLUNA'


In [118]:
refract_cluna_df_pol['operation'] = 'refraction'

In [119]:
refract_cluna_df_pol['hr'] = refract_cluna_df_pol.block_timestamp.str[:-9] + '00:00.000'
refract_cluna_df_pol['day'] = refract_cluna_df_pol.block_timestamp.str[:-13]

In [120]:
refract_cluna_df_pol['amount_signed'] = refract_cluna_df_pol.amount
refract_cluna_df_pol.head()

,block_timestamp,tx_id,user,action,amount,asset_given,asset_received,operation,hr,day,amount_signed
0,2022-02-14 13:24:17.166,349C9F2F540E2C0B39034F5B8D5BBC114E468C71150A6E19FE3FE09551D3461E,terra1xc6kttvedkf96f9rz0klc3d8ff7sfy9hjrxg6v,transfer_from,180.000000,cLUNA,yLUNA,refraction,2022-02-14 13:00:00.000,2022-02-14,180.000000
1,2022-02-14 16:29:56.862,D6644B621DD9A31DBD0800BC6D9D5889ED5EEB861DF7023519DA3E0B1AF41C7C,terra1jxaq9xnympm5l4yvccdayyara65uvphu6l8tva,transfer_from,0.018917,cLUNA,yLUNA,refraction,2022-02-14 16:00:00.000,2022-02-14,0.018917
2,2022-02-14 15:21:18.039,D5C7595087DBFEDFE0F7A31E424FDAEE004B63228617395A31EB755628AA8B6F,terra1dyu2rvykn5wkhp2jwa90a9fkqwk88z62mk47nf,transfer_from,265.542139,cLUNA,yLUNA,refraction,2022-02-14 15:00:00.000,2022-02-14,265.542139
3,2022-02-14 16:59:19.788,ED9B70CD5DC09AA1DA555929ADCC8D173C1E6C01CFECB62776B5DE639EC8B21D,terra10xt0yhdullk2n8st46dja4tazkrckkyvlsjfac,transfer_from,1.841609,cLUNA,yLUNA,refraction,2022-02-14 16:00:00.000,2022-02-14,1.841609
4,2022-02-14 15:33:56.849,6B6BE138D1F9C2079687DD316C5F1C7E9928E1C93314121CA68C771F674670F9,terra1mkhgc2tq33n90uaa5nc4yrhgp86xy79e8mtxz5,transfer_from,1.000000,cLUNA,yLUNA,refraction,2022-02-14 15:00:00.000,2022-02-14,1.000000


### Refract

In [121]:
refract_df['user'] = refract_df.apply(lambda row: row.from_ if row['0_action']=='bond_split' else row.to_, axis=1)

In [122]:
refract_df.shape

(8888, 16)

In [123]:
refract_df[refract_df.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']

,0_action,0_amount,1_amount,1_contract_address,2_amount,2_contract_address,3_contract_address,block_timestamp,event_index,from_,msg_index,to_,tx_id,bonded,minted,user
3077,bond_split,7.976801,7.976801,terra13zaagrrrxj47qjwczsczujlvnnntde7fdt0mau,7.976801,terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz,terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2,2022-02-14 17:40:44.210,4,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,0,terra1xw3h7jsmxvh6zse74e4099c6gl03fnmxpep76h,20C3745D6BEFF089670317C7445D3A2F79F588DE88EB3A078CEB4AA3B1AEF862,7.976801,7.976801,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q


In [124]:
refract_df_pol = refract_df[['block_timestamp','tx_id','user','0_action','0_amount']]
refract_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']

In [125]:
refract_df_pol['asset_given'] = refract_df_pol.apply(lambda row: 'LUNA' if row['action']=='bond_split' else 'yLUNA',axis=1)
refract_df_pol['asset_received'] = refract_df_pol.apply(lambda row: 'yLUNA' if row['action']=='bond_split' else 'LUNA',axis=1)


In [126]:
refract_df_pol['operation'] = 'refraction'

In [127]:
refract_df_pol[refract_df_pol.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']

,block_timestamp,tx_id,user,action,amount,asset_given,asset_received,operation
3077,2022-02-14 17:40:44.210,20C3745D6BEFF089670317C7445D3A2F79F588DE88EB3A078CEB4AA3B1AEF862,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,bond_split,7.976801,LUNA,yLUNA,refraction


In [128]:
refract_df_pol['hr'] = refract_df_pol.block_timestamp.str[:-9] + '00:00.000'
refract_df_pol['day'] = refract_df_pol.block_timestamp.str[:-13]

In [129]:
refract_df_pol.head()

,block_timestamp,tx_id,user,action,amount,asset_given,asset_received,operation,hr,day
0,2022-02-14 16:28:44.314,23F63DF850EBD1A94822B5DE5BD3B9208DEEAC7FFF1C25A9D5BCD8FBF3917E1F,terra1smusxgwactgq9w4566lgr0p4fvxdwuxew257u9,bond_split,216.498676,LUNA,yLUNA,refraction,2022-02-14 16:00:00.000,2022-02-14
1,2022-02-14 12:33:18.758,B6AD4BADF9CE783AB6BBDA4C4ADD01F663D079A61C09004C50FD8606D0C4D349,terra170sp9lhr9689lhwxv7hud5qgz26zlxvvnrl6kf,bond_split,12.000000,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14
2,2022-02-14 12:24:45.981,E0656127AB4575D129F1E5D42F5019F5129B1845D05D469ECE8B937C2B7CD9A6,terra1q9uurxw5cv4e8axqfluw2zxx0esl40z8pv2zj5,bond_split,8.745652,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14
3,2022-02-14 14:04:43.220,FBEEB7090427833FD276113CAE700793D7024A8A9E74A303E30C93CBBDF4CA61,terra1x5rutmkr9q6cvr4mhapy0x3aav9jq3qt8n8j0k,bond_split,2.774851,LUNA,yLUNA,refraction,2022-02-14 14:00:00.000,2022-02-14
4,2022-02-14 12:55:22.960,DE0CA803E9A11CF1A68D5FE84FC407071D6EB4AD16B65EB4DA4BDC9CFD8B8348,terra15p0qacpqxzdvmlprvds99wvlsa8c24e2fur7hg,bond_split,100.000000,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14


In [130]:
refract_df_pol['amount_signed'] = refract_df_pol.apply(lambda row: -row.amount 
                                                if row.action=='burn_from' else row.amount,axis=1)
refract_df_pol.head()

,block_timestamp,tx_id,user,action,amount,asset_given,asset_received,operation,hr,day,amount_signed
0,2022-02-14 16:28:44.314,23F63DF850EBD1A94822B5DE5BD3B9208DEEAC7FFF1C25A9D5BCD8FBF3917E1F,terra1smusxgwactgq9w4566lgr0p4fvxdwuxew257u9,bond_split,216.498676,LUNA,yLUNA,refraction,2022-02-14 16:00:00.000,2022-02-14,216.498676
1,2022-02-14 12:33:18.758,B6AD4BADF9CE783AB6BBDA4C4ADD01F663D079A61C09004C50FD8606D0C4D349,terra170sp9lhr9689lhwxv7hud5qgz26zlxvvnrl6kf,bond_split,12.000000,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14,12.000000
2,2022-02-14 12:24:45.981,E0656127AB4575D129F1E5D42F5019F5129B1845D05D469ECE8B937C2B7CD9A6,terra1q9uurxw5cv4e8axqfluw2zxx0esl40z8pv2zj5,bond_split,8.745652,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14,8.745652
3,2022-02-14 14:04:43.220,FBEEB7090427833FD276113CAE700793D7024A8A9E74A303E30C93CBBDF4CA61,terra1x5rutmkr9q6cvr4mhapy0x3aav9jq3qt8n8j0k,bond_split,2.774851,LUNA,yLUNA,refraction,2022-02-14 14:00:00.000,2022-02-14,2.774851
4,2022-02-14 12:55:22.960,DE0CA803E9A11CF1A68D5FE84FC407071D6EB4AD16B65EB4DA4BDC9CFD8B8348,terra15p0qacpqxzdvmlprvds99wvlsa8c24e2fur7hg,bond_split,100.000000,LUNA,yLUNA,refraction,2022-02-14 12:00:00.000,2022-02-14,100.000000


In [131]:
all_refreact = refract_df_pol.append(refract_cluna_df_pol)

In [132]:
daily_delta_rf = all_refreact.groupby('day').amount_signed.sum().reset_index()
daily_delta_rf = daily_delta_rf.sort_values(by='day')
daily_delta_rf['cumsum'] = daily_delta_rf.amount_signed.cumsum().apply(lambda x: round(x,2))
daily_delta_rf.columns = ['Time', 'Amount signed', 'Amount']
daily_delta_rf['Type'] = 'yLuna circulating'

In [133]:
domain = ['yLuna circulating']
range_ = ['lightblue']
max_date = daily_delta_rf['Time'].max()
mars_price_chart = alt.Chart(daily_delta_rf).mark_line(point = True).encode(
    x=alt.X('Time:T',scale=alt.Scale(domain=(daily_delta_rf.Time.min(),max_date))),
    y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,daily_delta_rf['Amount'].max()+100000))),
    color=alt.Color('Type:N', 
                sort=domain,
                scale=alt.Scale(domain=domain, range=range_),
                legend=alt.Legend(
                            orient='none',
                            padding=5,
                            legendY=0,
                            direction='horizontal')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'Amount:Q']
).properties(width=800).configure_view(strokeOpacity=0)
mars_price_chart

alt.Chart(...)

### yStaking

In [134]:
ystaking_df.head()

,0_action,0_amount,1_action,1_amount,block_timestamp,event_index,from_,hr,msg_index,to_,tx_id
0,send,2.889441,bond,2.889441,2022-02-16 02:18:56.229,1,terra1rvvhec2zgultp3uvjs278yzn0ts2wldmlwunax,2022-02-16 02:00:00.000,0,terra1p7jp8vlt57cf8qwazjg58qngwvarmszsamzaru,780D505D7942CAE5837F994EBCA9054AED03526A555501A28CB346849565F84C
1,send,10.022220,bond,10.022220,2022-02-16 10:41:33.123,1,terra1k8lssx4v42efe5mhplcwfer7vyljqq3qw3v856,2022-02-16 10:00:00.000,0,terra1p7jp8vlt57cf8qwazjg58qngwvarmszsamzaru,62748116DD64ECC0840DCF9463A426A13BA946D73226BBBA087BA2E4165BF7D0
2,send,73.713479,bond,73.713479,2022-02-16 20:18:54.710,1,terra1kn5a6yd9xnprygklgmpn753phaksne2qxmqcm3,2022-02-16 20:00:00.000,0,terra1p7jp8vlt57cf8qwazjg58qngwvarmszsamzaru,084252F31766579956BA283FD6EBD92D97A72AB29DC89E189AF5FBBBAFF580D2
3,send,0.171457,bond,0.171457,2022-02-16 16:36:19.873,1,terra1s78xphdlh9nc3xgncfk7zk9f5wak2wea6ljurw,2022-02-16 16:00:00.000,0,terra1p7jp8vlt57cf8qwazjg58qngwvarmszsamzaru,3C620714D1B97543B8AC5F886076407FA89F218291F12480B5546127DD5F5288
4,unbond,3478.000000,transfer,3478.000000,2022-02-16 10:57:40.568,1,terra1p7jp8vlt57cf8qwazjg58qngwvarmszsamzaru,2022-02-16 10:00:00.000,0,terra1n8utaly5st5du65fgaqawhrkvm0qrlfws9kymc,599EFDD53CC110C87B1E951AF0E50F8939E8018CD39D40A672794EDD5D32FC3F


In [135]:
ystaking_df['action'] = ystaking_df.apply(lambda row: row['0_action'] if row['0_action']=='unbond' else row['1_action'],axis=1)
ystaking_df['amount'] = ystaking_df.apply(lambda row: row['0_amount'] if row['0_action']=='unbond' else row['1_amount'],axis=1)
ystaking_df['user'] = ystaking_df.apply(lambda row: row['to_'] if row.action=='unbond' else row['from_'],axis=1)
ystaking_df['asset_given'] = ystaking_df.apply(lambda row: None if row.action=='unbond' else 'yLUNA',axis=1)
ystaking_df['asset_received'] = ystaking_df.apply(lambda row: 'yLUNA' if row.action=='unbond' else None,axis=1)
ystaking_df['day'] = ystaking_df.hr.str[:-13]


In [136]:
ystaking_df = ystaking_df[['block_timestamp','tx_id','hr','day','action','amount','user','asset_given','asset_received']]

In [137]:
ystaking_df

,block_timestamp,tx_id,hr,day,action,amount,user,asset_given,asset_received
0,2022-02-16 02:18:56.229,780D505D7942CAE5837F994EBCA9054AED03526A555501A28CB346849565F84C,2022-02-16 02:00:00.000,2022-02-16,bond,2.889441,terra1rvvhec2zgultp3uvjs278yzn0ts2wldmlwunax,yLUNA,None
1,2022-02-16 10:41:33.123,62748116DD64ECC0840DCF9463A426A13BA946D73226BBBA087BA2E4165BF7D0,2022-02-16 10:00:00.000,2022-02-16,bond,10.022220,terra1k8lssx4v42efe5mhplcwfer7vyljqq3qw3v856,yLUNA,None
2,2022-02-16 20:18:54.710,084252F31766579956BA283FD6EBD92D97A72AB29DC89E189AF5FBBBAFF580D2,2022-02-16 20:00:00.000,2022-02-16,bond,73.713479,terra1kn5a6yd9xnprygklgmpn753phaksne2qxmqcm3,yLUNA,None
3,2022-02-16 16:36:19.873,3C620714D1B97543B8AC5F886076407FA89F218291F12480B5546127DD5F5288,2022-02-16 16:00:00.000,2022-02-16,bond,0.171457,terra1s78xphdlh9nc3xgncfk7zk9f5wak2wea6ljurw,yLUNA,None
4,2022-02-16 10:57:40.568,599EFDD53CC110C87B1E951AF0E50F8939E8018CD39D40A672794EDD5D32FC3F,2022-02-16 10:00:00.000,2022-02-16,unbond,3478.000000,terra1n8utaly5st5du65fgaqawhrkvm0qrlfws9kymc,None,yLUNA
...,...,...,...,...,...,...,...,...,...
6715,2022-03-05 02:13:49.456,0C193964ACA70B32A1E64AC2E2E6EF7335D581AF1572CA516711BC484B720691,2022-03-05 02:00:00.000,2022-03-05,bond,3.226230,terra1c6x5qcq899232hjfg5anhtr2z22fj20tg0dfmg,yLUNA,None
6716,2022-03-05 01:51:41.578,94C8BC44AB46F8CC697B55C7CCBFE8E0EB9DB5342493FDEFF6985EF9A6CFF7C2,2022-03-05 01:00:00.000,2022-03-05,bond,183.076773,terra1cntd7rqu98aa788z0wmjgrfz8gsw728vazrf3x,yLUNA,None
6717,2022-03-05 01:53:26.126,407931822EE2AEF452E619DBC4FBDD4383FDAA8958D500463DA091AD4E9E55F5,2022-03-05 01:00:00.000,2022-03-05,bond,3.027776,terra1g4pcakhxe5wn509ck5g5m9mkxlax6svtxffz5x,yLUNA,None
6718,2022-03-05 12:16:02.064,3AACDF33CEB6EC2397B8D8C5AD129FB28EA1274C86EC62F5A5E50AD3BD8EA740,2022-03-05 12:00:00.000,2022-03-05,bond,0.170074,terra14fhssytrujau0mp40g4sctddm52qcgegp698up,yLUNA,None


In [138]:
ystaking_df[ystaking_df.user=='terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv']

,block_timestamp,tx_id,hr,day,action,amount,user,asset_given,asset_received
5617,2022-02-28 18:31:55.044,378D0A25C6077B47F455939F69FBB5C0F3D09871518CA7C91DFD5E0CB7BC5D41,2022-02-28 18:00:00.000,2022-02-28,bond,29.352984,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,yLUNA,None


In [139]:
ystaking_df['amount_signed'] = ystaking_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)

In [140]:
daily_delta_stk = ystaking_df.groupby('day').amount_signed.sum().reset_index()
daily_delta_stk = daily_delta_stk.sort_values(by='day')
daily_delta_stk['cumsum'] = daily_delta_stk.amount_signed.cumsum().apply(lambda x: round(x,2))
daily_delta_stk.columns = ['Time', 'Amount signed', 'Amount']
daily_delta_stk['Type'] = 'yLuna staked'

In [141]:
def get_max_domain_date(df, time_field, n_hours):
    if((pd.Timestamp(df[time_field].max()) - 
                pd.Timestamp(df[time_field].min())).total_seconds()/3600 < n_hours):
        max_date = (pd.Timestamp(df[time_field].min()) + pd.to_timedelta(n_hours, unit='h')).strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        max_date = df[time_field].max()
    return max_date

In [142]:
domain = ['yLuna staked']
range_ = ['#f8936d']
max_date = daily_delta_stk['Time'].max()
mars_price_chart = alt.Chart(daily_delta_stk).mark_line(point = True).encode(
    x=alt.X('Time:T',scale=alt.Scale(domain=(daily_delta_stk.Time.min(),max_date))),
    y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,daily_delta_stk['Amount'].max()+100000))),
    color=alt.Color('Type:N', 
                sort=domain,
                scale=alt.Scale(domain=domain, range=range_),
                legend=alt.Legend(
                            orient='none',
                            padding=5,
                            legendY=0,
                            direction='horizontal')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'Amount:Q']
).properties(width=800).configure_view(strokeOpacity=0)
mars_price_chart

alt.Chart(...)

## All deltas

In [145]:
all_deltas = daily_delta_rf.append(daily_delta_stk).append(daily_delta_lp)

In [151]:
domain = ['yLuna staked','yLuna circulating','yLuna LP']
range_ = ['#f8936d','lightblue','green']
max_date = all_deltas['Time'].max()
mars_price_chart = alt.Chart(all_deltas).mark_line(point = True).encode(
    x=alt.X('Time:T',scale=alt.Scale(domain=(all_deltas.Time.min(),max_date))),
    y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,all_deltas['Amount'].max()+100000))),
    color=alt.Color('Type:N', 
                sort=domain,
                scale=alt.Scale(domain=domain, range=range_),
                legend=alt.Legend(
                            orient='none',
                            padding=5,
                            legendY=0,
                            direction='horizontal')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'Type:N','Amount:Q']
).properties(width=800).properties(height=600).configure_view(strokeOpacity=0)
mars_price_chart

alt.Chart(...)